# Day 1

In [3]:
turns = [int(i.replace("L", "-").replace("R", "")) for i in open("data/day1").read().strip().split("\n")]

sum([1 if (50+sum(turns[:i]))%100==0 else 0 for i in range(1, len(turns))])

1097

In [4]:
val = 50
counter = 0

for i in turns:
    if abs(i)//100 != 0:
        counter += abs(i)//100
        if i < 0:
            i -= (abs(i)//100)*-100
        else:
            i %= 100
    val += i
    if (abs(val) != abs(i)) and (val < 0 or val > 100):
        counter+= 1
    val %= 100
    if val == 0:
        counter += 1

counter

7101

# Day 2

In [1]:
%%timeit 

ranges = [list(map(int, i.split("-"))) for i in open("data/day2").read().replace("\n", "").split(",")]

invalid = 0
for rng in ranges:
    for i in range(rng[0], rng[1]+1):
        strint = str(i)
        if len(strint)%2 == 0:
            if strint[:len(strint)//2] == strint[len(strint)//2:]:
                invalid += int(strint)

invalid

213 ms ± 5.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
ranges = [list(map(int, i.split("-"))) for i in open("data/day2").read().replace("\n", "").split(",")]

invalid = 0
for rng in ranges:
    for i in range(rng[0], rng[1]+1):
        strint = str(i)
        for j in range(1, len(strint)//2+1):
            result = set(strint.split(strint[:j]))
            if len(result) == 1:
                invalid += i
                break
invalid

1.22 s ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [1]:
# second alternative to part 2, because regex is very powerful


# Day 3

In [2]:
input = open("data/day3").read().strip().split("\n")

total = 0
for i in input:
    digitmax = 0
    digits = list(map(int, i))
    for j, val in enumerate(digits):
        for l, val2 in enumerate(digits):
            if j != l and j < l:
                current = val * 10 + val2
                if current > digitmax:
                    digitmax = current
    total += digitmax
total

16858

In [3]:
input = open("data/test").read().strip().split("\n")

import re

total = 0
for i in input:
    currentmax = 0
    digits_remaining = 12
    highest_digit = 9
    while digits_remaining > 0:
        item = str(highest_digit) + "\\d" + "{" + str(digits_remaining-1) + "}"
        blah = re.search(item, i)
        if blah:
            currentmax += highest_digit * (10 ** (digits_remaining-1))
            i = i[i.index(str(highest_digit))+1:]
            digits_remaining -= 1
            highest_digit = 9
        else:
            highest_digit -= 1
    total += currentmax

total

3121910778619

# Day 4

In [1]:
positions = open("data/day4").read().strip().split("\n")

def perimeter(y, x, positions):
    if positions[y][x] != '@':
        return False
    toilet_rolls = 0
    left = max(0, x-1)
    right = min(x+1, len(positions[0])-1)
    up = max(0, y-1)
    down = min(y+1, len(positions)-1)

    for i in range(up, down+1):
        for j in range(left, right+1):
            if positions[i][j] == "@":
                toilet_rolls += 1
    toilet_rolls-= 1

    if toilet_rolls < 4:
        return True
    else:
        return False

counter = 0
for i in range(len(positions)):
    for j in range(len(positions[0])):
        if perimeter(i, j, positions):
            counter += 1

counter

1419

In [2]:
positions = open("data/day4").read().strip().split("\n")

def perimeter(y, x, positions):
    if positions[y][x] != '@':
        return False
    toilet_rolls = 0
    left = max(0, x-1)
    right = min(x+1, len(positions[0])-1)
    up = max(0, y-1)
    down = min(y+1, len(positions)-1)

    for i in range(up, down+1):
        for j in range(left, right+1):
            if positions[i][j] == "@":
                toilet_rolls += 1
    toilet_rolls-= 1

    if toilet_rolls < 4:
        return True
    else:
        return False


def clean(positions, counter):
    for i in range(len(positions)):
        for j in range(len(positions[0])):
            if perimeter(i, j, positions):
                counter += 1
                positions[i] = positions[i][:j] + '.' + positions[i][j+1:]
    return positions, counter

counter = 0
while True:
    positions, new_counter = clean(positions, counter)
    if new_counter == counter:
        break
    else:
        counter = new_counter 
counter

8739